# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "output_data/cities.csv"
weather = pd.read_csv(file)
weather.set_index('City Name', inplace=True)
weather.drop(columns=['Unnamed: 0'], inplace=True)
weather.reset_index()

,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Alamosa,1608350725,-105.87,37.47,15.80,92,3.36,90,US
1,Carutapera,1608350725,-46.02,-1.20,77.67,86,5.53,6,BR
2,Rincon,1608350724,-81.24,32.30,41.00,80,5.82,1,US
3,Qaanaaq,1608350725,-69.36,77.48,-1.91,84,0.96,14,GL
4,Bluff,1608350725,168.33,-46.60,64.99,93,1.99,100,NZ
...,...,...,...,...,...,...,...,...,...
705,Jalu,1608350801,21.55,29.03,52.66,82,7.70,56,LY
706,Verāval,1608350801,70.37,20.90,72.18,47,11.16,0,IN
707,Bairiki Village,1608350801,172.98,1.33,86.00,74,11.41,20,KI
708,Bentiu,1608350801,29.83,9.23,64.51,31,4.14,9,SS


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
locations = weather[["Lat", "Long"]].astype(float)
weight = weather['Humid']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal = weather[weather.eval('Temp>=65 & Temp < 90 & (Humid < 40) & clouds <= 50')]
ideal.reset_index(inplace=True)
ideal

,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Esperance,1608350728,121.90,-33.87,77.00,33,9.17,0,AU
1,Busselton,1608350730,115.33,-33.65,79.39,36,10.58,0,AU
2,Geraldton,1608350732,114.60,-28.77,87.80,33,19.46,0,AU
3,Dinguiraye,1608350740,-10.72,11.30,68.50,33,2.42,47,GN
4,Christchurch,1608350628,172.63,-43.53,82.40,23,10.29,45,NZ
5,Pāsighāt,1608350749,95.33,28.07,69.19,38,2.24,3,IN
6,Broken Hill,1608350756,141.43,-31.95,78.80,13,8.05,7,AU
7,Say,1608350768,2.37,13.10,71.60,30,1.12,0,NE
8,Goundam,1608350771,-3.67,16.41,69.04,24,9.48,23,ML
9,Malakal,1608350771,31.66,9.53,69.22,29,6.96,43,SS


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"
# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
#Utilize iterrows to loop through rows in hotel workbook. 
for index, row in ideal.iterrows():
    lat = row['Lat']
    long = row['Long']
    city = row['City Name']
    #append the lat and long from each row and add to 
    params["location"] = f'{lat},{long}'

    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        ideal.loc[index, "Hotel"] = results[0]['name']
    except :
        pass
    time.sleep(1)
        
print("------------")

Retrieving Results for Index 0: Esperance.


C:\Users\jonrs\anaconda3\envs\case_python\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jonrs\anaconda3\envs\case_python\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Retrieving Results for Index 1: Busselton.
Retrieving Results for Index 2: Geraldton.
Retrieving Results for Index 3: Dinguiraye.
Retrieving Results for Index 4: Christchurch.
Retrieving Results for Index 5: Pāsighāt.
Retrieving Results for Index 6: Broken Hill.
Retrieving Results for Index 7: Say.
Retrieving Results for Index 8: Goundam.
Retrieving Results for Index 9: Malakal.
Retrieving Results for Index 10: Inverell.
------------


In [12]:
ideal.dropna(how='any', inplace=True)
ideal

C:\Users\jonrs\anaconda3\envs\case_python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country,Hotel
0,Esperance,1608350728,121.90,-33.87,77.00,33,9.17,0,AU,Esperance Clearwater Motel Apartments
1,Busselton,1608350730,115.33,-33.65,79.39,36,10.58,0,AU,Esplanade Hotel
2,Geraldton,1608350732,114.60,-28.77,87.80,33,19.46,0,AU,Ocean Centre Hotel
4,Christchurch,1608350628,172.63,-43.53,82.40,23,10.29,45,NZ,Quality Hotel Elms
5,Pāsighāt,1608350749,95.33,28.07,69.19,38,2.24,3,IN,Donyi Hango Camp Homestay And Resort
6,Broken Hill,1608350756,141.43,-31.95,78.80,13,8.05,7,AU,Royal Exchange Hotel
9,Malakal,1608350771,31.66,9.53,69.22,29,6.96,43,SS,Nile Palace Hotel
10,Inverell,1608350800,151.12,-29.78,84.99,37,1.01,43,AU,Inverell Terrace Motor Lodge


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal.iterrows()]
locations = ideal[["Lat", "Long"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))